In [20]:
import minsearch
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables from .env
load_dotenv()

api_key = os.getenv("GOOGLE_GEMINI_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

In [21]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
    

In [36]:
from elasticsearch import Elasticsearch
# Creating index
es_client = Elasticsearch(
    "http://localhost:9200")
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name="faq-document"
es_client.indices.create(index=index_name, body=index_settings)


BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [faq-document/9PXSJcc1RWKC73mrBLkz-Q] already exists')

In [ ]:
documents=[];
for course_info in raw_doc: 
    for doc in course_info['documents']:
        doc['course']=course_info['course']
        documents.append(doc);

In [ ]:
from tqdm.auto import tqdm 

for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

In [ ]:
# Q1. build hash version -  73f7594ea00db50aa7e941e151a5b3985f01e364

# curl http://localhost:9200/

# {
#   "name": "cdce3272b42d",
#   "cluster_name": "docker-cluster",
#   "cluster_uuid": "NPnoINGmRQKXpmqxEzI7vA",
#   "version": {
#     "number": "9.0.1",
#     "build_flavor": "default",
#     "build_type": "docker",
#     "build_hash": "73f7594ea00db50aa7e941e151a5b3985f01e364",
#     "build_date": "2025-04-30T10:07:41.393025990Z",
#     "build_snapshot": false,
#     "lucene_version": "10.1.0",
#     "minimum_wire_compatibility_version": "8.18.0",
#     "minimum_index_compatibility_version": "8.0.0"
#   },
#   "tagline": "You Know, for Search"
# }

In [ ]:
# Q2. Which function do you use for adding your data to elastic?

# insert
# index  (Ans)
# put
# add

<!-- # Q3. Searching
Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

84.50
64.50
44.50
24.50
Look at the _score field. -->  

In [23]:
# Q3. Searching
# Now let's search in our index.

# We will execute a query "How do execute a command on a Kubernetes pod?".

# Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

# What's the score for the top ranking result?

# 84.50
# 64.50
# 44.50 (Ans)
# 24.50
# Look at the _score field.

# Ans: '_score': 44.50556 , Please note that in the elastic_search I returned the response not the response["hits"]["hits"]  to get this below response

def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    return response

elastic_search("How do execute a command on a Kubernetes pod?")

ObjectApiResponse({'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'faq-document', '_id': 'lQ5ub5cBCvdFXF-rIokP', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'faq-document', '_id': 'JA5ub5cBCvdFXF-rPYob', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dash

In [25]:
# Q4. Filtering
# Now ask a different question: "How do copy a file to a Docker container?".

# This time we are only interested in questions from machine-learning-zoomcamp.

# Return 3 results. What's the 3rd question returned by the search engine?

# How do I debug a docker container?
# How do I copy files from a different folder into docker container’s working directory? 
# How do Lambda container images work?
# How can I annotate a graph?

# Ans:  'question': 'How do I copy files from a different folder into docker container’s working directory?'

def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])
    
    return result_docs

response=elastic_search("How do copy a file to a Docker container?")
response[2]  #  for 3rd result 

{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I copy files from a different folder into docker container’s working directory?',
 'course': 'machine-learning-zoomcamp'}

In [61]:
# Q5. Building a prompt  
# What's the length of the resulting prompt? (use the len function)

# 946
# 1446 (Ans)
# 1946
# 2446
# Ans:1451 which is close to 1446. Therefore 2nd option will be a correct option

results = elastic_search("How do copy a file to a Docker container?")
context_template = ''

for data in results:
    context_template += f"Q: {data['question']}\nA: {data['text']}\n\n"

context_template=context_template.strip().replace("\t","")

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


prompt = prompt_template.format(
    question="How do I execute a command in a running docker container?",
    context=context_template
)

print(len(prompt))

1451


In [64]:
# Q6: TOkens 

# Use the encode function. How many tokens does our prompt have?
# 120
# 220
# 320
# 420

# Ans: Since 354 is close to 320. So 320 is the ans

# Count tokens
token_count = model.count_tokens(prompt)

print("Total tokens:", token_count.total_tokens)


Total tokens: 354


In [80]:
# Bonus: generating the answer (ungraded)

response = model.generate_content(prompt)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "To execute a command in a running Docker container, first find the container ID using `docker ps`. Then, execute the command using `docker exec -it <container-id> bash`.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.12416525376148713
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 354,
        "candidates_token_count": 39,
        "total_token_count": 393
      },
      "model_version": "gemini-1.5-flash"
    }),
)

In [78]:
# Bonus: calculating the costs (ungraded)
# As of now pricing (per million) for gemini-1.5-flash api:
# Input price: 	$0.075, prompts <= 128k tokens
#                 $0.15, prompts > 128k tokens
# Output Price:  $0.30, prompts <= 128k tokens
#                $0.60, prompts > 128k tokens

input_token_count=response.usage_metadata.prompt_token_count 
output_token_count=response.usage_metadata.candidates_token_count 


# Cost calculation
input_cost = (input_token_count / 1_000_000) * 0.075
output_cost = (output_token_count / 1_000_000) * 0.30

total_cost = input_cost + output_cost
print(total_cost)

3.855e-05
